In [87]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers, models, optimizers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# Now that we know the image dimensions
train_data_dir = '../data/project3/data_all_modified/data_split/train'
# Number of images we want to process at once
batch_size = 64

# Target image size (128 px by 128 px)
img_height = 128
img_width = 128
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [88]:
test_data_dir = '../data/project3/data_all_modified/data_split/test'

batch_size = 1

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4265 files belonging to 2 classes.


In [89]:
from keras import layers
from keras import models
import pandas as pd

model_mod_lenet5 = models.Sequential()
# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 2: Convolutional layer with 32 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 3: Convolutional layer with 64 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 4: Convolutional layer with 128 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 5: Convolutional layer with 128 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# flattening the output of the conv layer after max pooling 
model_mod_lenet5.add(layers.Flatten())
#Dropout Layer
model_mod_lenet5.add(layers.Dropout(0.2))
# Layer 8: Fully connected layer with 120 neurons
model_mod_lenet5.add(layers.Dense(512, activation='relu'))

model_mod_lenet5.add(layers.Dense(2, activation='sigmoid'))

model_mod_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

# Generating the summary of the model
model_mod_lenet5.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 126, 126, 6)       168       
                                                                 
 max_pooling2d_45 (MaxPooli  (None, 63, 63, 6)         0         
 ng2D)                                                           
                                                                 
 conv2d_46 (Conv2D)          (None, 61, 61, 32)        1760      
                                                                 
 max_pooling2d_46 (MaxPooli  (None, 30, 30, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_47 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_47 (MaxPooli  (None, 14, 14, 64)       

In [90]:
import tensorflow as tf
model = tf.keras.models.load_model('Mod_LeNet5.keras')

In [91]:
import requests

In [92]:
iterator = iter(test_rescale_ds)
import numpy as np
from PIL import Image
l = np.array(Image.open('../data/project3/data_all_modified/damage/-93.528502_30.987438.jpeg')).tolist()

# make the POST request passing the single test case as the `image` field:
rsp = requests.post("http://172.17.0.1:5000/proj_models/Mod_LeNet5/v1", json={"image":l})

In [93]:
rsp.json()

{'result': [[0.9861195087432861, 0.021355213597416878]]}